# Homo LR Demo

In this demo, we use the homo_logistic_regression(https://github.com/FederatedAI/FATE/tree/master/examples/federatedml-1.x-examples/homo_logistic_regression example. It is:
* Homogeneous federated machine learning example, which both parties share same attributes but different samples;
* Use breast cancer data original from Kaggle: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data
* For easy to demo, both party we use the same FATE cluster: 10000. But the underlayer is the same, each side threat the collborated party go through the network to another party. 

Step 0. Prepare the libary to manage federated machine learning.

In [1]:
import json
import time
import os
import requests

import fml_manager
from fml_manager import *

manager = fml_manager.FMLManager()

Step 1. Upload the guest, host and test data. Because we use same cluster for this demo, we load all data in same NOTEBOOK. If we use another party for host, the host data should load in the NOTEBOOK of that party.

In [3]:
response = manager.load_data(url='./data/breast_homo_guest.csv', namespace='homo_breast_guest', table_name='homo_breast_guest', work_mode=1, head=1, partition=10)
output = json.loads(response.content)
print(output)
guest_job_id = output["jobId"]
guest_query_condition = {
    'job_id':guest_job_id
}
response = manager.load_data(url='./data/breast_homo_host.csv', namespace='homo_breast_host', table_name='homo_breast_host', work_mode=1, head=1, partition=10)
output = json.loads(response.content)
host_job_id = output["jobId"]
host_query_condition = {
    'job_id':host_job_id
}
response = manager.load_data(url='./data/breast_homo_test.csv', namespace='homo_breast_test', table_name='homo_breast_test', work_mode=1, head=1, partition=10)
output = json.loads(response.content)
test_job_id = output["jobId"]
test_query_condition = {
    'job_id':test_job_id
}


manager.query_job_status(guest_query_condition)
manager.query_job_status(host_query_condition)
manager.query_job_status(host_query_condition)


{'data': {'board_url': 'http://fateboard:8080/index.html#/dashboard?job_id=2020063006322784993833&role=local&party_id=0', 'job_dsl_path': '/data/projects/fate/python/jobs/2020063006322784993833/job_dsl.json', 'job_runtime_conf_path': '/data/projects/fate/python/jobs/2020063006322784993833/job_runtime_conf.json', 'logs_directory': '/data/projects/fate/python/logs/2020063006322784993833', 'namespace': 'homo_breast_guest', 'table_name': 'homo_breast_guest'}, 'jobId': '2020063006322784993833', 'retcode': 0, 'retmsg': 'success'}
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: success
Status: success
Status: success
success,success,success
Success


Step 2. Create the steps DSL and configuration of each step for training.

In [4]:
# dsl
data_io = ComponentBuilder()\
    .with_name('dataio_0')\
    .with_module('DataIO')\
    .with_input_data('arg.train_data')\
    .with_output_data('train')\
    .with_output_model('dataio').build()
        

homo_lr = ComponentBuilder()\
    .with_name('homo_lr_0')\
    .with_module('HomoLR')\
    .with_input_train_data('dataio_0.train')\
    .with_output_data('train')\
    .with_output_model('homolr').build()

evaluation = ComponentBuilder()\
    .with_name('evaluation_0')\
    .with_module('Evaluation')\
    .with_input_data('homo_lr_0.train')\
    .with_output_data('evaluate').build()

dsl = PiplineBuilder()\
    .with_components(
        data_io, 
        hetero_lr, 
        evaluation).build()

# Configuration
initiator = InitiatorBuilder()\
    .with_role("guest")\
    .with_party_id(10000).build()


job_parameters = JobParametersBuilder()\
    .with_work_mode(1).build()

role = RoleBuilder()\
    .with_guest(10000)\
    .with_host(10000)\
    .with_arbiter(10000).build()

eval_config = {
       "need_run": [False]
 }

role_parameters = RoleParametersBuilder()\
    .with_guest_train_data(namespaces=['homo_breast_guest'], names=['homo_breast_guest'])\
    .with_host_train_data(namespaces=['homo_breast_host'], names=['homo_breast_host'])\
    .with_host_module_config(modules=['evalution_0'], configs=[eval_config]).build()


homo_lr_params = {
            "penalty": "L2",
            "optimizer": "sgd",
            "eps": 1e-5,
            "alpha": 0.01,
            "max_iter": 10,
            "converge_func": "diff",
            "batch_size": 500,
            "learning_rate": 0.15,
            "decay": 1,
            "decay_sqrt": True,
            "init_param": {
                "init_method": "zeros"
            },
            "encrypt_param": {
                "method": "Paillier"
            },
            "cv_param": {
                "n_splits": 4,
                "shuffle": True,
                "random_seed": 33,
                "need_cv": False
            }
        }
dotaio_config = {
            "with_label": True,
            "label_name": "y",
            "label_type": "int",
            "output_format": "dense"
        },

algorithm_parameters = AlgorithmParametersBuilder()\
    .with_module_config(modules=['homo_lr_0', 'dataio_0'], configs=[homo_lr_params, dotaio_config]).build()

config = ConfigBuilder()\
    .with_initiator(initiator)\
    .with_job_parameters(job_parameters)\
    .with_role(role)\
    .with_role_parameters(role_parameters)\
    .with_algorithm_parameters(algorithm_parameters).build()


Step 3. Submit the training job to GUEST cluster. And it will notify and bring up the HOST cluster and train together. 

In [5]:
response = manager.submit_job(dsl.to_dict(),config.to_dict())
manager.prettify(response, verbose=True)
stdout = json.loads(response.content)
jobId = stdout["jobId"]
query_condition = {
    'job_id':jobId
}

model_id, model_version = '', ''
manager.query_job_status(query_condition)

manager.prettify(response, verbose=True)
output = json.loads(response.content)
model_id, model_version = output["data"]["model_info"]["model_id"], output["data"]["model_info"]["model_version"]

Success!
{
    "data": {
        "board_url": "http://fateboard:8080/index.html#/dashboard?job_id=2020063006362564981036&role=guest&party_id=9999",
        "job_dsl_path": "/data/projects/fate/python/jobs/2020063006362564981036/job_dsl.json",
        "job_runtime_conf_path": "/data/projects/fate/python/jobs/2020063006362564981036/job_runtime_conf.json",
        "logs_directory": "/data/projects/fate/python/logs/2020063006362564981036",
        "model_info": {
            "model_id": "arbiter-9999#guest-9999#host-9999#model",
            "model_version": "2020063006362564981036"
        }
    },
    "jobId": "2020063006362564981036",
    "retcode": 0,
    "retmsg": "success"
}
Status: waiting
Status: waiting
Status: waiting
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: run

In [6]:
response = manager.model_output('guest', '10000', model_id, model_version, 'homo_lr_0.homolr:HomoLogisticRegression')

{'metadata': 'CgJMMhHxaOOItfjkPhl7FK5H4XqEPyIDc2dkMPQDOTMzMzMzM8M/QApKBGRpZmZQAlgB', 'parameters': 'CAoSUP4QBQQ+79o/Y+ScSvKa1j+RTreM/XfUPyk/T2KsNtM/ik0jT2po0j+lMOUXo9zRPwWjqz1MetE/3VWUfhkz0T8KB2etSv7QP5ymdO1A1tA/Ig0KAngwEbJDSoWA/sG/Ig0KAngxEcob/dudLr6/Ig0KAngyEWYZ+hcRpMG/Ig0KAngzEfWv/7Rc+r+/Ig0KAng0EbDD/W9RW7m/Ig0KAng1Eb6OMlppJri/Ig0KAng2EbIEbULVqru/Ig0KAng3ERQQVXbn+MG/Ig0KAng4EW6/Nt4yu7W/Ig0KAng5EQJZbnGtrKW/Ig4KA3gxMBEsGFvBSI3AvyIOCgN4MTERZZM+KKAGur8iDgoDeDEyEdGPglYzgMC/Ig4KA3gxMxHa0QRjolK+vyIOCgN4MTQR9tqVWW+Fr78iDgoDeDE1ETwimmbYbLO/Ig4KA3gyMBG7wwtUoXG0vyIOCgN4MTYRIGmQvHuRur8iDgoDeDIxEdqP/oYJfnY/Ig4KA3gxNxFSVtQreM/AvyIOCgN4MjIRzQmY7no3sr8iDgoDeDIzEcwFGjerVbK/Ig4KA3gxOBHDXhlmJQ2rvyIOCgN4MjQRoPtqyzc7iD8iDgoDeDE5EXOSYAuFHaM/Ig4KA3gyNRGNsnK5wSqBvyIOCgN4MjYRna/Rsrf0Gj8iDgoDeDI3EUcfkMZaT6a/Ig4KA3gyOBHXUPahnWidPyIOCgN4MjkRXdrDc3/ooD8pCAy8iYha4T8yAngwMgJ4MTICeDIyAngzMgJ4NDICeDUyAng2MgJ4NzICeDgyAng5MgN4MTAyA3gxMTIDeDEyMgN4MTMyA3gxNDIDeDE1MgN4MTYyA3gxNzIDeDE4MgN4MTkyA3gyMDIDeDIxMgN4MjIyA3gyMzID

And we can try offline prediction feature. Prediction also need both parts participant.

In [7]:
is_vertical = False
initiator_party_role = 'guest'
initiator_party_id = '10000'
work_mode = 1
federated_roles = {
        'guest': [10000],
        'host': [10000],
        'arbiter': [10000]
}
guest_data_name = 'homo_breast_test'
guest_data_namespace = 'homo_breast_test'
host_data_name = 'homo_breast_test'
host_data_namespace = 'homo_breast_test'

response = manager.offline_predict_on_dataset(is_vertical, initiator_party_role, initiator_party_id, work_mode, model_id, model_version, federated_roles, guest_data_name, guest_data_namespace, host_data_name, host_data_namespace)
print(response.text)

{"data":{"board_url":"http://fateboard:8080/index.html#/dashboard?job_id=202005070653465729886&role=guest&party_id=10000","job_dsl_path":"/data/projects/fate/python/jobs/202005070653465729886/job_dsl.json","job_runtime_conf_path":"/data/projects/fate/python/jobs/202005070653465729886/job_runtime_conf.json","logs_directory":"/data/projects/fate/python/logs/202005070653465729886","model_info":{"model_id":"arbiter-10000#guest-10000#host-10000#model","model_version":"202005070651022620445"}},"jobId":"202005070653465729886","retcode":0,"retmsg":"success"}



The result can be checked in FATE-Board.